In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o")

In [12]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You give short and best practice python code."),
    ("user", "{input}")
])

chain = prompt | llm 
ai_message = chain.invoke({"input": "Write a python function that uses regex to extract emails (including instances of 'name at email dot com') from a text. Use pytest to write useful unit tests."})

In [14]:
print(ai_message.content)

Certainly! Below is a Python function that uses regex to extract emails, including instances formatted as 'name at email dot com', from a given text. Additionally, I've included unit tests using `pytest`.

```python
import re

def extract_emails(text):
    # Regex pattern to match standard emails and 'name at email dot com' format
    pattern = re.compile(r'\b[\w.-]+@[\w.-]+\.\w{2,4}\b|\b[\w.-]+ at [\w.-]+ dot \w{2,4}\b', re.IGNORECASE)
    matches = pattern.findall(text)
    
    # Convert 'name at email dot com' to 'name@email.com'
    emails = [re.sub(r' at ', '@', re.sub(r' dot ', '.', match, flags=re.IGNORECASE), flags=re.IGNORECASE) for match in matches]
    
    return emails

# Unit tests using pytest
def test_extract_emails():
    text = """
    Here are some emails: test.email@example.com, another.email@domain.org.
    Also, consider this format: user at example dot com, admin at domain dot net.
    """
    expected_emails = [
        'test.email@example.com',
        'anothe